In [283]:
# imports
import requests
import os 
import pandas as pd
import json
import pprint
import sqlite3

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [142]:
# Set the key
fsq_api_key = os.environ['FOURSQUARE_API_KEY']

In [210]:
# Set the url and parameters that will be used in our request
base_url = 'https://api.foursquare.com/v3/places/search'
radius = 1000
ll = '44.232344187881836%2C-76.48559630147832' # This is the latitude/longitude for Kingston Centre in Kingston, ON, replaced comma with '%2C
limit = 50 # This is the maximum number of entries per request
fields = 'rating%2Cprice%2Cpopularity%2Cname%2Cdistance%2Clocation%2Ccategories'
params = f'?ll={ll}&radius={radius}&fields={fields}&limit={limit}'

# I did not specify a venue type (e.g. 'bar') to get all bars, restaurants and POIs in this radius 
# Combine  
url = base_url+params

In [211]:
# get request
headers = {
    "accept": "application/json",
    "Authorization": fsq_api_key
}

response = requests.get(url, headers=headers)

In [212]:
# check response status
response.status_code

200

In [214]:
# save the response as a json file to prevent from sending repeated requests

# !!! DO NOT RUN OR IT WILL OVERWRITE THE SAVED FILE !!! 

#with open('fsq_response.json', 'w') as f:
#    f.write(response.text)

Parse through the response to get the POI details you want (rating, name, location, etc)

In [215]:
# let's now work only with our fsq_response.txt file to avoid sending unnecessary requests
with open('fsq_response.json') as f:
    fsq_data = json.load(f)
pprint.pprint(fsq_data)

{'context': {'geo_bounds': {'circle': {'center': {'latitude': 44.232344187881836,
                                                  'longitude': -76.48559630147832},
                                       'radius': 1000}}},
 'results': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/giftshop_',
                                       'suffix': '.png'},
                              'id': 17089,
                              'name': 'Gift Store'}],
              'distance': 85,
              'location': {'address': '165 Princess St',
                           'country': 'CA',
                           'cross_street': 'btw Wellington & King',
                           'formatted_address': '165 Princess St (btw '
                                                'Wellington & King), Kingston '
                                                'ON K7L 1A9',
                           'locality': 'Kingston',
                           'neighborhood': ['Syden

From the response above, we can see that we have a nested dictionary. The details we want to keep are within the results key, which on the other hand is a list of dictionaries with each POI. Within each entry, we may want to keep the category name, POI name and location (formatted address). Let's explore this structure below.

In [240]:
# number of POIs
len(fsq_data['results'])

# POI Name
#poi_name = fsq_data['results'][0]['name']
#poi_name

# POI address
#poi_address = fsq_data['results'][0]['location']['address']
#poi_address

# POI type
#poi_type = fsq_data['results'][0]['categories'][0]['name']
#poi_type

# POI rating
#poi_rating = fsq_data['results'][0]['rating']
#poi_rating

# POI price
#poi_price = fsq_data['results'][4]['price']
#poi_price

# POI popularity
#poi_popularity = fsq_data['results'][4]['popularity']
#poi_popularity

50

In [202]:
# Not all entries have ratings or prices, so we need to add an exception to handle that
# The example below shows how some ratings are missing.

rating_key = 'rating'
price_key = 'price' # will be used later

for i in range(len(fsq_data['results'])):
    if rating_key in fsq_data['results'][i]:
        print('Rating is available')
        
    else:
        print('Rating is unavailable')

Rating is available
Rating is available
Rating is available
Rating is available
Rating is available
Rating is available
Rating is available
Rating is available
Rating is available
Rating is available
Rating is available
Rating is available
Rating is available
Rating is available
Rating is available
Rating is available
Rating is available
Rating is unavailable
Rating is unavailable
Rating is available
Rating is available
Rating is unavailable
Rating is available
Rating is unavailable
Rating is unavailable
Rating is available
Rating is unavailable
Rating is available
Rating is available
Rating is available
Rating is available
Rating is available
Rating is available
Rating is unavailable
Rating is available
Rating is available
Rating is unavailable
Rating is available
Rating is available
Rating is available
Rating is unavailable
Rating is available
Rating is available
Rating is available
Rating is unavailable
Rating is available
Rating is unavailable
Rating is unavailable
Rating is availa

Loop through the entries to create a dict which we will import to pandas dataframe

In [ ]:
# Empty lists that we will add to the dict
names = []
addresses = []
types = []
ratings = []
prices = []
popularity = []

In [219]:
for i in range(len(fsq_data['results'])):
    # add POI name to names list
    names.append(fsq_data['results'][i]['name'])
    
    # add POI address to addresses list
    addresses.append(fsq_data['results'][i]['location']['address'])
    
    # add POI type to types list
    types.append(fsq_data['results'][i]['categories'][0]['name'])
    
    # add POI rating to ratings list - need to account for missing ratings
    if rating_key in fsq_data['results'][i]:
        ratings.append(fsq_data['results'][i]['rating'])
    else:
        ratings.append('NA')
        
    # add POI price info to prices list - also use the if statement to handle missing values
    if price_key in fsq_data['results'][i]:
        prices.append(fsq_data['results'][i]['price'])
    else:
        prices.append('NA')

    # add POI popularity rating to popularity list
    popularity.append(fsq_data['results'][i]['popularity'])

# Print lists to check 
#print(names)
#print(addresses)
#print(types)
#print(ratings)
#print(prices)
#print(popularity)

['Minotaur Games & Gifts', 'Dollarama', 'Crave Coffee House & Bakery', "Menchie's", 'Stone City Ales', 'Amadeus Cafe', 'Hwa-Ki', 'The Screening Room', "Balzac's Coffee", 'Grecos Grill & Wine Bar', 'Lululemon Athletica', 'Shoppers Drug Mart', 'Urban Outfitters', 'NORTHSIDE espresso + kitchen', "Harper's", 'Golden Rooster Delicatessen', 'Copper Penny', 'Kingston Frameworks', 'Plaza Hotel', 'Ta Ke Sushi', 'Starbucks', 'Racquet Science', "Megalo's Restaurants", 'Union Kitchen + Cocktails', 'Northern Chinese Barbeque', 'Mango Thai & Pan-Asian Cuisine', 'True North Tattoo', 'Score Pizza', 'The Alibi', 'The Toucan', 'Saigon Delights', 'Chez Piggy', 'Geneva Crepe Cafe', 'Andrei Master Tailors', 'CIBC', 'Le Chien Noir', "Graham's Pharmacy", 'Sima Sushi', 'Tango Nuevo', 'The Iron Duke on Wellington', "St Paul's Anglican Church", "Sir John's Public House", 'Pan Chancho Bakery', 'Musiikki', 'Overstock', "The Queen's Inn", 'Cambodiana Restaurant', 'Novel Idea', "Cooke's Fine Foods and Coffee", 'Kin

In [220]:
# Create a dict with the lists
fsq_dict = {
    'Name': names,
    'Address': addresses,
    'Type': types,
    'Rating': ratings,
    'Price': prices,
    'Popularity': popularity
}

Put your parsed results into a DataFrame

In [270]:
# create a dataframe from the fsq_dict variable, rounding to two decimals for readability
fsq_df = pd.DataFrame(fsq_dict).round(2)
fsq_df

,Name,Address,Type,Rating,Price,Popularity
0,Minotaur Games & Gifts,165 Princess St,Gift Store,8.4,NA,0.86
1,Dollarama,176 Princess St,Discount Store,7.3,NA,0.93
2,Crave Coffee House & Bakery,166 Princess St,Bakery,8.0,1,0.96
3,Menchie's,241 Princess St,Frozen Yogurt Shop,7.7,1,0.82
4,Stone City Ales,275 Princess St,Brewery,9.0,2,0.93
5,Amadeus Cafe,170 Princess St,Café,6.9,2,0.92
6,Hwa-Ki,269 Princess St,Sushi Restaurant,7.9,2,0.29
7,The Screening Room,120 Princess St,Indie Movie Theater,8.3,NA,0.87
8,Balzac's Coffee,251 Princess St,Café,7.3,1,0.94
9,Grecos Grill & Wine Bar,167 Princess St,Diner,6.6,2,0.91


In [268]:
# Explore the data by sorting based on popularity
pop = fsq_df[['Name', 'Type', 'Popularity']].nlargest(5, 'Popularity', keep='all')
pop

,Name,Type,Popularity
11,Shoppers Drug Mart,Drugstore,0.97
42,Pan Chancho Bakery,Bakery,0.97
2,Crave Coffee House & Bakery,Bakery,0.96
8,Balzac's Coffee,Café,0.94
15,Golden Rooster Delicatessen,Caterer,0.94
20,Starbucks,Coffee Shop,0.94
29,The Toucan,Pub,0.94
40,St Paul's Anglican Church,Church,0.94


In [282]:
# Let's look at some statistics for popularity
# This rating goes from 0 to 1, with one being most popular
print(fsq_df['Popularity'].max())
print(fsq_df['Popularity'].min())
print(fsq_df['Popularity'].mean())

0.97
0.07
0.8036000000000001


# Yelp

Send a request with the same location paramaters (location, radius, etc)

In [187]:
# Set the key
yelp_api_key = os.environ['YELP_API_KEY']

In [188]:
# Set the GET url according to Yelp API documentation 
# First, define host p
host = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'Bearer {}'.format(yelp_api_key)}

# This is the latitude, should be a decimal 
lat = 44.232344187881836

# This is the longitude, should be a decimal 
lon = -76.48559630147832

# Radius should be an int
radius = 1000 

# Limit should be an int
limit = 50

# Parameters for our request
params = f'?latitude={lat}&longitude={lon}&radius={radius}&limit={limit}'


# For comparison's sake I didn't specify the type of venue I am looking for
# Combine  
url = host+params
print(url)

https://api.yelp.com/v3/businesses/search?latitude=44.232344187881836&longitude=-76.48559630147832&radius=1000&limit=50


In [189]:
# get request
yelp_response = requests.get(url, headers=headers)

In [190]:
yelp_response.status_code

200

In [30]:
# save the response as a json file to prevent from sending repeated requests

# !!! DON'T RUN OR IT WILL OVERWRITE THE SAVED FILE !!!

#with open('yelp_response.json', 'w') as f:
#    f.write(yelp_response.text)

Parse through your result and get POI details

In [225]:
with open('yelp_response.json') as f:
    yelp_data = json.load(f)
pprint.pprint(yelp_data)

{'businesses': [{'alias': 'wooden-heads-kingston',
                 'categories': [{'alias': 'pizza', 'title': 'Pizza'},
                                {'alias': 'mediterranean',
                                 'title': 'Mediterranean'},
                                {'alias': 'italian', 'title': 'Italian'}],
                 'coordinates': {'latitude': 44.2290169,
                                 'longitude': -76.48106688543417},
                 'display_phone': '+1 613-549-1812',
                 'distance': 516.8355833542643,
                 'id': 'sZ3glyC2gtZL0Hk08RFkWQ',
                 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/cT37Xk72f91rQzVLW_nsCw/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '192 Ontario Street',
                              'address2': '',
                              'address3': '',
                              'city': 'Kingston',
                              'country': 'CA',
                   

Again, we have a nested dictionary as the response. We can see that all POIs are kept inside a list within the dict key 'businesses'. Below, let's play around to find points of interest. In this case, our response is more rich in details, so we can select more information on each POI.

In [84]:
# number of POIs
#len(yelp_data['businesses'])

# POI Name
#poi_name = yelp_data['businesses'][0]['name']
#poi_name

# POI address
#poi_address = yelp_data['businesses'][0]['location']['address1']
#poi_address

# POI type
#poi_type = yelp_data['businesses'][0]['categories'][0]['title']
#poi_type

# POI rating
#poi_rating = yelp_data['businesses'][0]['rating']
#poi_rating

# POI price
#poi_price = yelp_data['businesses'][0]['price']
#poi_price

In [227]:
# Empty lists that we will add to the yelp_dict
y_names = []
y_addresses = []
y_types = []
y_ratings = []
y_prices = []

for i in range(len(yelp_data['businesses'])):
    # add POI name to names list
    y_names.append(yelp_data['businesses'][i]['name'])
    
    # add POI address to addresses list
    y_addresses.append(yelp_data['businesses'][i]['location']['address1'])
    
    # add POI type to types list
    y_types.append(yelp_data['businesses'][i]['categories'][0]['title'])
    
    # add POI rating to ratings list
    y_ratings.append(yelp_data['businesses'][i]['rating'])
    
    # add POI price info to prices list - using the same if statement to handle missing values
    if price_key in yelp_data['businesses'][i]:
        y_prices.append(yelp_data['businesses'][i]['price'])
    else:
        y_prices.append('NA')
        
print(y_names)
print(y_addresses)
print(y_types)
print(y_ratings)
print(y_prices)

['Wooden Heads', 'Tango Nuevo', 'Pan Chancho Bakery', "Harper's", "Greco's Grill and Wine Bar", 'Wok-In Restaurant', 'Sima Sushi', 'NORTHSIDE Espresso + Kitchen', 'Mio Gelato', 'Amadeus Cafe', "Dianne's Fish Shack & Smokehouse", 'Chez Piggy Restaurant', "Sally's Roti Shop", 'Red House', 'Black Dog Tavern', 'Olivea', "Balzac's Coffee Roasters", 'Score Pizza', 'Ali Baba', 'Kame Sushi and Izakaya', 'The Cambodian Village', 'Geneva Crepe Cafe', 'Gangnam Style', 'Copper Penny', 'Sushibar Da', 'Mango Restaurant', 'Sushi Ya Zen', 'Royal Angkor Restaurant', 'Podonamu', 'CRAVE Coffee House & Bakery', "Peter's Place", 'The Keg Steakhouse + Bar - Kingston', 'Cambodiana Restaurant', 'Apsara Angkor', 'Thai House Cuisine', 'Atomica', 'i-tea', 'AquaTerra', "Pat's Restaurant", 'Casa', 'The Pilot House', "Choi's Roll", 'Sunflower Chinese Restaurant', 'Meet Cuisine', 'Golden Rooster Delicatessen', 'White Mountain', "Jack Astor's", 'Northern Chinese Barbeque', 'The Rustic Spud Restaurant & Catering', 'Fl

In [228]:
# Create a dict with the lists
yelp_dict = {
    'Name': y_names,
    'Address': y_addresses,
    'Type': y_types,
    'Rating': y_ratings,
    'Price': y_prices
}

Put your parsed results into a DataFrame

In [229]:
# create a dataframe from the fsd_dict variable
yelp_df = pd.DataFrame(yelp_dict)
yelp_df

,Name,Address,Type,Rating,Price
0,Wooden Heads,192 Ontario Street,Pizza,4.0,$$
1,Tango Nuevo,331 King Street E,Tapas/Small Plates,4.5,$$$
2,Pan Chancho Bakery,44 Princess Street,Bakeries,4.0,$$
3,Harper's,93 Princess Street,Burgers,4.0,$$
4,Greco's Grill and Wine Bar,167 Princess Street,Greek,4.0,$$
5,Wok-In Restaurant,30 Montreal St,Cambodian,5.0,$
6,Sima Sushi,66 Princess St,Japanese,4.0,$$
7,NORTHSIDE Espresso + Kitchen,281 Princess Street,Cafes,4.5,$$
8,Mio Gelato,178 Ontario Street,Ice Cream & Frozen Yogurt,4.5,$$
9,Amadeus Cafe,170 Princess Street,German,4.5,$$


In [251]:
# Explore the data by grouping based on venue type
yelp_df.groupby('Type').agg(n_venues = ('Type', 'count')).nlargest(5, 'n_venues', keep='all')

,n_venues
Type,
Cambodian,5
Japanese,4
Chinese,3
Italian,3
Thai,3


# Google (stretch)

Use the same process as the first two APIs

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [132]:
connection = sqlite3.connect("/home/eduardo/lighthouse-data-notes/mini-project-II/data/miniproject2_db.sqlite")

In [233]:
# First, let's add the fsq_df to our database
# Create connection to the database
connection = sqlite3.connect("/home/eduardo/lighthouse-data-notes/mini-project-II/data/miniproject2_db.sqlite")

# Push the dataframe to sql 
fsq_df.to_sql("fsq", connection, if_exists="replace", index=False)

# Close connection with the database when done
#connection.close()

50

In [234]:
# Now, let's add the yelp_df to our database
# Create connection to the database
connection = sqlite3.connect("/home/eduardo/lighthouse-data-notes/mini-project-II/data/miniproject2_db.sqlite")

# Push the dataframe to sql 
yelp_df.to_sql("yelp", connection, if_exists="replace", index=False)

# Close connection with the database when done
#connection.close()

50

In [235]:
# To obtain information from our database, we will use pandas .read_sql() method
# First, let's test to see if our tables were created correctly
pd.read_sql('SELECT * FROM fsq', connection)

,Name,Address,Type,Rating,Price,Popularity
0,Minotaur Games & Gifts,165 Princess St,Gift Store,8.4,NA,0.86
1,Dollarama,176 Princess St,Discount Store,7.3,NA,0.93
2,Crave Coffee House & Bakery,166 Princess St,Bakery,8.0,1,0.96
3,Menchie's,241 Princess St,Frozen Yogurt Shop,7.7,1,0.82
4,Stone City Ales,275 Princess St,Brewery,9.0,2,0.93
5,Amadeus Cafe,170 Princess St,Café,6.9,2,0.92
6,Hwa-Ki,269 Princess St,Sushi Restaurant,7.9,2,0.29
7,The Screening Room,120 Princess St,Indie Movie Theater,8.3,NA,0.87
8,Balzac's Coffee,251 Princess St,Café,7.3,1,0.94
9,Grecos Grill & Wine Bar,167 Princess St,Diner,6.6,2,0.91


In [236]:
# Testing the yelp table
pd.read_sql('SELECT * FROM yelp', connection)

,Name,Address,Type,Rating,Price
0,Wooden Heads,192 Ontario Street,Pizza,4.0,$$
1,Tango Nuevo,331 King Street E,Tapas/Small Plates,4.5,$$$
2,Pan Chancho Bakery,44 Princess Street,Bakeries,4.0,$$
3,Harper's,93 Princess Street,Burgers,4.0,$$
4,Greco's Grill and Wine Bar,167 Princess Street,Greek,4.0,$$
5,Wok-In Restaurant,30 Montreal St,Cambodian,5.0,$
6,Sima Sushi,66 Princess St,Japanese,4.0,$$
7,NORTHSIDE Espresso + Kitchen,281 Princess Street,Cafes,4.5,$$
8,Mio Gelato,178 Ontario Street,Ice Cream & Frozen Yogurt,4.5,$$
9,Amadeus Cafe,170 Princess Street,German,4.5,$$


Get the top 10 restaurants according to their rating

In [140]:
# To get the top 10 restaurants, we will need an ORDER BY in our query. This will be done with each table.
# Yelp table
query = '''
    SELECT
        y.Name, 
        y.Rating
    FROM
        yelp AS y
    ORDER BY 
        y.Rating DESC
    LIMIT
        10
'''

pd.read_sql(query, connection)

,Name,Rating
0,Wok-In Restaurant,5.0
1,Balzac's Coffee Roasters,5.0
2,Tango Nuevo,4.5
3,NORTHSIDE Espresso + Kitchen,4.5
4,Mio Gelato,4.5
5,Amadeus Cafe,4.5
6,Sally's Roti Shop,4.5
7,Score Pizza,4.5
8,Kame Sushi and Izakaya,4.5
9,The Cambodian Village,4.5


In [239]:
# FSQ table - remove missing ('NA') rating information
query = '''
    SELECT
        f.Name, 
        f.Rating
    FROM
        fsq AS f
    WHERE
        f.Rating IS NOT 'NA'
    ORDER BY 
        f.Rating DESC
    LIMIT
        10
'''

pd.read_sql(query, connection)

,Name,Rating
0,Pan Chancho Bakery,9.1
1,Stone City Ales,9.0
2,Tango Nuevo,8.9
3,Minotaur Games & Gifts,8.4
4,Chez Piggy,8.4
5,Sir John's Public House,8.4
6,The Screening Room,8.3
7,Harper's,8.3
8,The Toucan,8.3
9,Le Chien Noir,8.1


# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).